In [1]:
import pandas as pd

# Load the first 38 rows of the CSV file (NIH ids)
df = pd.read_csv('/Users/sanjaymandal/Downloads/count_10mer_180genome.csv', nrows=38,index_col=0)

# Display the DataFrame
df.head()

,AAAAAAAAAA,AAAAAAAAAC,AAAAAAAAAG,AAAAAAAAAT,AAAAAAAACA,AAAAAAAACC,AAAAAAAACG,AAAAAAAACT,AAAAAAAAGA,AAAAAAAAGC,...,TTTTAAAAAA,TTTTACAAAA,TTTTAGAAAA,TTTTATAAAA,TTTTCAAAAA,TTTTCCAAAA,TTTTCGAAAA,TTTTGAAAAA,TTTTGCAAAA,TTTTTAAAAA
NIH-003-1,46,10,18,6,3,34,14,5,6,28,...,0,1,2,1,7,5,3,2,3,0
NIH-004-1,38,16,7,5,9,27,13,5,7,29,...,0,1,3,3,7,3,3,4,0,0
NIH-004-2,168,21,8,9,11,29,13,5,7,29,...,1,1,3,3,7,3,3,5,0,1
NIH-004-3,45,17,7,2,8,26,14,6,6,30,...,0,1,3,3,7,3,3,4,0,0
NIH-004-4,54,20,11,5,14,27,15,6,9,31,...,1,1,3,3,7,3,3,4,0,0


In [2]:
# Using IMR MIC data(S=0, R=1)
df['MCInt']=[0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0]

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold,StratifiedKFold
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')



In [4]:
X = df.drop('MCInt', axis=1) 
y = df['MCInt'] 

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  
X_test = scaler.transform(X_test) 

In [6]:
# Handling non-normal distribution
power_transformer = PowerTransformer(method='yeo-johnson', standardize=False)  
X_train = power_transformer.fit_transform(X_train)
X_test = power_transformer.transform(X_test)

In [7]:
from sklearn.feature_selection import SelectKBest, f_classif

# Example: Select the top K features based on f_classif score
selector = SelectKBest(score_func=f_classif, k=min(100, X_train.shape[1]))  
X_train = selector.fit_transform(X_train, y_train)  
X_test = selector.transform(X_test)               

print("Shape of X_train after feature selection:", X_train.shape)
print("Shape of X_test after feature selection:", X_test.shape)

Shape of X_train after feature selection: (30, 100)
Shape of X_test after feature selection: (8, 100)


In [8]:
# Using XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42) 

In [9]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [10]:
y_pred = xgb_model.predict(X_test)

In [11]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy (XGBoost):", accuracy)
print("Classification Report (XGBoost):\n", classification_report(y_test, y_pred))

Accuracy (XGBoost): 1.0
Classification Report (XGBoost):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         2

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



In [12]:
print(y_pred)
print(y_test)

[0 0 0 0 0 1 0 1]
NIH-006-1    0
NIH-019-1    0
NIH-004-4    0
NIH-012-2    0
NIH-026-1    0
NIH-025-3    1
NIH-004-6    0
NIH-025-4    1
Name: MCInt, dtype: int64


In [13]:

# To predict in the ncbi dataset
df1 = pd.read_csv('/Users/sanjaymandal/Downloads/count_10mer_180genome.csv', skiprows=range(1, 40), nrows=13, index_col=0)
df1.head()

,AAAAAAAAAA,AAAAAAAAAC,AAAAAAAAAG,AAAAAAAAAT,AAAAAAAACA,AAAAAAAACC,AAAAAAAACG,AAAAAAAACT,AAAAAAAAGA,AAAAAAAAGC,...,TTTTAAAAAA,TTTTACAAAA,TTTTAGAAAA,TTTTATAAAA,TTTTCAAAAA,TTTTCCAAAA,TTTTCGAAAA,TTTTGAAAAA,TTTTGCAAAA,TTTTTAAAAA
GCA-000186185-1,1,6,9,1,6,20,10,4,2,22,...,0,1,2,1,6,4,3,3,2,0
GCA-000508285-1,4,11,12,1,7,32,13,2,2,37,...,0,2,3,1,6,5,3,4,2,0
GCA-000517225-1,2,6,10,1,4,30,15,2,3,35,...,0,1,2,1,5,3,4,3,2,0
GCA-000967095-1,0,7,9,3,3,26,15,1,2,34,...,0,1,2,1,8,3,3,1,2,0
GCA-001038045-1,2,9,9,2,5,29,16,4,4,32,...,0,1,3,1,6,4,3,5,3,0


In [14]:
ncb_test= scaler.transform(df1)

In [15]:
ncb_test =power_transformer.transform(ncb_test)

In [16]:
ncb_test = selector.transform(ncb_test)

In [17]:
ncb_test.shape

(13, 100)

In [18]:
# Probable predictions
pred_ncb = xgb_model.predict(ncb_test)
print(df1.index)
print("NCB predictions:", pred_ncb)

Index(['GCA-000186185-1', 'GCA-000508285-1', 'GCA-000517225-1',
       'GCA-000967095-1', 'GCA-001038045-1', 'GCA-001051055-1',
       'GCA-001276315-1', 'GCA-001298795-1', 'GCA-001298895-1',
       'GCA-001298915-1', 'GCA-001298935-1', 'GCA-001299015-1',
       'GCA-001299035-1'],
      dtype='object')
NCB predictions: [0 1 0 1 0 0 1 0 1 1 1 0 1]


In [19]:
# Get probability estimates for the ncb set
probabilities = xgb_model.predict_proba(ncb_test)

# Display the probability estimates
for i, prob in enumerate(probabilities):
    print(f"Sample {i+1}: Class 0 Probability = {prob[0]:.4f}, Class 1 Probability = {prob[1]:.4f}")

Sample 1: Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 2: Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 3: Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 4: Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 5: Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 6: Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 7: Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 8: Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 9: Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 10: Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 11: Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 12: Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 13: Class 0 Probability = 0.1039, Class 1 Probability = 0.8961


In [20]:
# Compare predictions with probability estimates
for i, (pred_ncb, probabilities) in enumerate(zip(pred_ncb, probabilities)):
    print(f"Sample {i+1}: Predicted Class = {pred_ncb}, Class 0 Probability = {probabilities[0]:.4f}, Class 1 Probability = {probabilities[1]:.4f}")

Sample 1: Predicted Class = 0, Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 2: Predicted Class = 1, Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 3: Predicted Class = 0, Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 4: Predicted Class = 1, Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 5: Predicted Class = 0, Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 6: Predicted Class = 0, Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 7: Predicted Class = 1, Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 8: Predicted Class = 0, Class 0 Probability = 0.9480, Class 1 Probability = 0.0520
Sample 9: Predicted Class = 1, Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 10: Predicted Class = 1, Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 11: Predicted Class = 1, Class 0 Probability = 0.1039, Class 1 Probability = 0.8961
Sample 1

In [23]:
results_df = pd.DataFrame({'NIH_Actual': y_test, 'NIH_Predicted': y_pred})




excel_file = "imr_prediction_results_NIH.xlsx"  


results_df.to_excel(excel_file, index=True)  

print(f"Prediction results exported to {excel_file}")

Prediction results exported to imr_prediction_results_NIH.xlsx


In [33]:
results1_df = pd.DataFrame(pred_ncb, df1.index)




excel_file = "imr_prediction_results_NCB.xlsx"  


results1_df.to_excel(excel_file, index=True)  

print(f"Prediction results exported to {excel_file}")

Prediction results exported to imr_prediction_results_NCB.xlsx
